In [1]:
import dash                     # pip install dash
from dash.dependencies import Input, Output, State
from dash import dcc
from dash import html
import plotly.express as px     # pip install plotly==5.2.2
import pandas as pd 

In [2]:
df = pd.read_excel("E:\Work & Study\MBA\T4\BDDA2\Project\Analytics Dashboard\EU_Superstore_Dataset.xlsx")

In [3]:
df.head()

,Row ID,Order ID,Order Date,Dispatch Date,Delivery Mode,Customer ID,Customer Name,Segment,City,State/Province,Country/Region,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,11,ES-2020-2510515,20-06-2020,20-06-2020,Same Day,LH-17155,Logan Haushalter,Consumer,Le Bouscat,Aquitaine-Limousin-Poitou-Charentes,France,Central,OFF-LA-10002805,Office Supplies,Labels,"Hon Color Coded Labels, Alphabetical",25.260,2,0.00,0.480
1,12,ES-2020-2510515,20-06-2020,20-06-2020,Same Day,LH-17155,Logan Haushalter,Consumer,Le Bouscat,Aquitaine-Limousin-Poitou-Charentes,France,Central,OFF-AP-10002330,Office Supplies,Appliances,"Hamilton Beach Stove, Silver",2443.905,5,0.10,760.305
2,13,ES-2020-2510515,20-06-2020,20-06-2020,Same Day,LH-17155,Logan Haushalter,Consumer,Le Bouscat,Aquitaine-Limousin-Poitou-Charentes,France,Central,OFF-AR-10000505,Office Supplies,Art,"Binney & Smith Pens, Easy-Erase",12.210,1,0.00,0.000
3,14,ES-2020-2510515,20-06-2020,20-06-2020,Same Day,LH-17155,Logan Haushalter,Consumer,Le Bouscat,Aquitaine-Limousin-Poitou-Charentes,France,Central,TEC-PH-10002898,Technology,Phones,"Samsung Smart Phone, Full Size",2167.296,4,0.15,790.416
4,15,ES-2020-2510515,20-06-2020,20-06-2020,Same Day,LH-17155,Logan Haushalter,Consumer,Le Bouscat,Aquitaine-Limousin-Poitou-Charentes,France,Central,OFF-ST-10001413,Office Supplies,Storage,"Rogers Folders, Blue",138.105,5,0.10,-12.345


In [4]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div([
    html.H1("Analytics Dashboard of EU Superstore Dataset (Dash Plotly)", style={"textAlign":"center"}),
    html.Hr(),
    html.P("Choose Product Category:"),
    html.Div(html.Div([
        dcc.Dropdown(id='category-type', clearable=False,
                     value="Office Supplies",
                     options=[{'label': x, 'value': x} for x in
                              df["Category"].unique()]),
    ],className="two columns"),className="row"),

    html.Div(id="output-div", children=[]),
])
@app.callback(Output(component_id="output-div", component_property="children"),
              Input(component_id="category-type", component_property="value"),
)
def make_graphs(category_chosen):
   
    # HISTOGRAM
    df_hist = df[df["Category"]==category_chosen]
    fig_hist = px.histogram(df_hist, x="Sub-Category")
    fig_hist.update_xaxes(categoryorder="total descending")
    # STRIP CHART
    fig_strip = px.strip(df_hist, x="Quantity", y="Sub-Category")

    # SUNBURST
    df_sburst = df.dropna(subset=['Category'])
    df_sburst = df_sburst[df_sburst["Sub-Category"].isin(["Labels", "Appliances", "Art", "Phones", "Machines"])]
    fig_sunburst = px.sunburst(df_sburst, path=["Category", "Sub-Category", "Quantity"])

    # Empirical Cumulative Distribution
    df_ecdf = df[df["Category"].isin(["Office Supplies","Technology"])]
    fig_ecdf = px.ecdf(df_ecdf, x="Sales", color="Category")

    # LINE CHART
    df_line = df
    df_line = df_line.groupby(
        ["Dispatch Date", "Category"]).size().reset_index(name="count")
    df_line.head()
    fig_line = px.line(df_line, x="Dispatch Date", y="count",
                       color="Category", markers=True)

    return [
        html.Div([
            html.Div([dcc.Graph(figure=fig_hist)], className="six columns"),
            html.Div([dcc.Graph(figure=fig_strip)], className="six columns"),
        ], className="row"),
        html.H2("All Categories", style={"textAlign":"center"}),
        html.Hr(),
        html.Div([
            html.Div([dcc.Graph(figure=fig_sunburst)], className="six columns"),
            html.Div([dcc.Graph(figure=fig_ecdf)], className="six columns"),
        ], className="row"),
        html.Div([
            html.Div([dcc.Graph(figure=fig_line)], className="twelve columns"),
        ], className="row"),
    ]


In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
